In [1]:
import pandas as pd 
import numpy as np
import os
import random
file="/home/renmengxing/audioRec/audio_data/train.csv"
audio_dir="/home/renmengxing/audioRec/audio_data/train"
df = pd.read_csv(file)


In [ ]:
label_counts_series = df['label'].value_counts()
label_counts_dict = label_counts_series.to_dict()
print(label_counts_dict,len(label_counts_dict))
print(label_counts_dict.keys())
# for label, count in label_counts_dict.items():


In [ ]:
from IPython.display import Audio, display

for index, data in df.iterrows(): # 使用 iterrows() 来正确地遍历 DataFrame
    audio_name = data['fname']
    audio_path = os.path.join(audio_dir, audio_name)
    label = data['label']
    print(audio_path, label)
    display(Audio(audio_path, autoplay=False))
    
    break


In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# CSV 文件路径 (原始数据集)
file="/home/renmengxing/audioRec/audio_data/train.csv"

# 你的音频文件所在的根目录
# 请务必修改为你的音频文件所在的实际目录，该目录应包含 train.csv 中 fname 列指向的所有 .wav 文件
audio_dir = "/home/renmengxing/audioRec/audio_data/train/" # <-- **请务必修改为你的音频文件目录**

# 定义输出文件的路径
output_dir = "/home/renmengxing/audioRec/" # 可以根据需要修改输出目录
os.makedirs(output_dir, exist_ok=True) # 确保输出目录存在

output_train_csv = os.path.join(output_dir, "data_train.csv")
output_val_csv = os.path.join(output_dir, "data_val.csv") # 通常验证集命名为 val 或 validation

print("正在加载数据集...")
try:
    df = pd.read_csv(file)
    df = df.dropna() # 清理数据，删除含有NaN值的行
    print(f"原始数据集大小: {len(df)} 条")

    # 为每个音频文件名构建绝对路径
    df['absolute_audio_path'] = df['fname'].apply(lambda x: os.path.abspath(os.path.join(audio_dir, x)))

    # 检查所有生成的路径是否存在
    missing_files = [path for path in df['absolute_audio_path'] if not os.path.exists(path)]
    if missing_files:
        print(f"\n警告: 发现 {len(missing_files)} 个音频文件不存在于指定路径。")
        # 如果你想移除这些缺失文件的行，可以取消注释下面这行
        # df = df[~df['absolute_audio_path'].isin(missing_files)]
        # print(f"已移除 {len(missing_files)} 条缺失文件的记录，剩余 {len(df)} 条。")
    else:
        print("所有音频文件路径均已确认存在。")

    # 拆分数据集为训练集和验证集 (8:2 比例)
    train_df, val_df = train_test_split(
        df,
        test_size=0.2,    # 20% 作为验证集
        random_state=42,  # 设置随机种子，保证每次拆分结果一致
        stratify=df['label'] # 按照标签进行分层抽样
    )

    print(f"\n数据集拆分完成：")
    print(f"训练集大小: {len(train_df)} 条")
    print(f"验证集大小: {len(val_df)} 条")

    # 将拆分后的 DataFrame 写入新的 CSV 文件
    # index=False 避免将 DataFrame 的索引写入到 CSV 文件中
    train_df.to_csv(output_train_csv, index=False)
    val_df.to_csv(output_val_csv, index=False)

    print(f"\n训练集已写入: {output_train_csv}")
    print(f"验证集已写入: {output_val_csv}")

    print("\n写入文件的示例内容 (训练集前5行):")
    # 为了验证，可以再次读取并打印部分内容
    check_train_df = pd.read_csv(output_train_csv)
    print(check_train_df.head())


except FileNotFoundError:
    print(f"错误: CSV文件 '{file}' 未找到。请检查路径。")
except Exception as e:
    print(f"处理数据集或写入文件时发生错误: {e}")

In [ ]:
import os
import pandas as pd

# 定义包含测试音频文件的目录
audio_dir = "/home/renmengxing/audioRec/audio_data/test" # <-- 请务必检查并确认这个路径正确

# 定义输出 CSV 文件的路径和文件名
# 假设你想把 test.csv 放在 /home/renmengxing/audioRec/ 目录下
output_csv_path = "/home/renmengxing/audioRec/test.csv"

# 用于存储文件名和绝对路径的列表
file_data = []

print(f"正在扫描目录: {audio_dir} 查找 .wav 文件...")

# 检查音频目录是否存在
if not os.path.exists(audio_dir):
    print(f"错误: 指定的音频目录 '{audio_dir}' 不存在。请检查路径。")
else:
    # 遍历指定目录下的所有文件和子目录
    for root, dirs, files in os.walk(audio_dir):
        for file in files:
            # 检查文件是否以 .wav 结尾（不区分大小写）
            if file.lower().endswith(".wav"):
                # 获取文件的绝对路径
                absolute_path = os.path.abspath(os.path.join(root, file))
                # 提取文件名 (不包含路径)
                file_name = file
                file_data.append({'fname': file_name, 'absolute_audio_path': absolute_path})
                # print(f"找到文件: {file_name} -> {absolute_path}") # 可以取消注释这行来实时查看进度

    if not file_data:
        print(f"在目录 '{audio_dir}' 中没有找到任何 .wav 文件。")
    else:
        # 将收集到的数据转换为 DataFrame
        test_df = pd.DataFrame(file_data)

        # 将 DataFrame 写入 CSV 文件
        # index=False 避免将 DataFrame 的索引写入到 CSV 文件中
        test_df.to_csv(output_csv_path, index=False)

        print(f"\n成功生成 test.csv 文件。")
        print(f"文件位置: {output_csv_path}")
        print(f"文件中包含 {len(test_df)} 条记录。")

        # 打印文件前几行以供预览
        print("\n生成文件的前5行内容:")
        print(test_df.head())

In [ ]:
import csv
import os

# Define the input and output file paths
input_csv_file = '/home/renmengxing/audioRec/audio_data/example.csv'
output_text_file = 'text.txt'
prefix = '/media/zhjk/audio_llm/rmx/funasr/AudioCLS/audio_data/test'

def process_csv_and_generate_text_file(csv_path, output_path, prefix_str):
    """
    Reads 'fname' from a CSV, adds a prefix, and writes to a text file.

    Args:
        csv_path (str): The path to the input CSV file.
        output_path (str): The path to the output text file.
        prefix_str (str): The prefix to add to each filename.
    """
    processed_fnames = []
    try:
        with open(csv_path, mode='r', newline='', encoding='utf-8') as infile:
            reader = csv.DictReader(infile) # Use DictReader to access columns by name
            
            # Check if 'fname' column exists
            if 'fname' not in reader.fieldnames:
                print(f"Error: CSV file '{csv_path}' does not contain a column named 'fname'.")
                return

            for row in reader:
                original_fname = row['fname']
                # Use os.path.join for robust path concatenation
                full_path = os.path.join(prefix_str, original_fname)
                processed_fnames.append(full_path)
        
        with open(output_path, mode='w', encoding='utf-8') as outfile:
            for line in processed_fnames:
                outfile.write(line + '\n')
        
        print(f"Successfully processed '{csv_path}' and generated '{output_path}'.")
        print(f"Total lines written: {len(processed_fnames)}")

    except FileNotFoundError:
        print(f"Error: Input CSV file not found at '{csv_path}'")
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the function
process_csv_and_generate_text_file(input_csv_file, output_text_file, prefix)

In [4]:
from tqdm import tqdm
import librosa


import pandas as pd 
import numpy as np
import os
import random

file="/home/renmengxing/audioRec/test.csv"
audio_dir="/home/renmengxing/audioRec/audio_data/test"
df = pd.read_csv(file)
# df[:5]

# 初始化最长和最短时长
min_duration = float('inf')  # 初始化为无穷大，确保任何真实时长都会比它小
max_duration = 0.0           # 初始化为0，确保任何真实时长都会比它大
total_audio_count = 0
processed_audio_count = 0

print("正在计算音频时长，请稍候...")

# 添加一列来存储音频时长
df['duration'] = 0.0

# 使用 tqdm 包装 iterrows() 以显示进度条
for index, row in tqdm(df.iterrows(), total=len(df), desc="计算音频时长"):
    audio_name = row['fname']
    audio_path = os.path.join(audio_dir, audio_name)
    
    if not os.path.exists(audio_path):
        print(f"警告: 音频文件不存在，已跳过: {audio_path}")
        continue
    
    try:
        # 加载音频，只读取其采样率和总样本数，避免加载整个波形到内存
        # sr=None 表示使用音频文件的原始采样率
        y, sr = librosa.load(audio_path, sr=None)
        
        # 计算音频时长（秒）
        duration = librosa.get_duration(y=y, sr=sr)
        
        # 更新最长和最短时长
        if duration < min_duration:
            min_duration = duration
        if duration > max_duration:
            max_duration = duration
            
        # 将时长写入 DataFrame
        df.at[index, 'duration'] = duration
        
        processed_audio_count += 1
        
    except Exception as e:
        print(f"处理音频 {audio_path} 时发生错误: {e}")
        continue

print("\n时长计算完成！")
print(f"成功处理音频文件数量: {processed_audio_count} / {len(df)}")

if processed_audio_count > 0:
    print(f"最短音频时长: {min_duration:.2f} 秒")
    print(f"最长音频时长: {max_duration:.2f} 秒")
else:
    print("没有音频文件被成功处理。请检查 'df' 和 'audio_dir' 的设置以及文件是否存在。")

# 保存更新后的 DataFrame 到新的 CSV 文件
output_file = "/home/renmengxing/audioRec/audio_data/test_with_duration.csv"
df.to_csv(output_file, index=False)
print(f"音频时长已写入文件: {output_file}")

正在计算音频时长，请稍候...


计算音频时长:   0%|          | 0/1598 [00:00<?, ?it/s]

计算音频时长: 100%|██████████| 1598/1598 [00:02<00:00, 538.91it/s]


时长计算完成！
成功处理音频文件数量: 1598 / 1598
最短音频时长: 0.30 秒
最长音频时长: 29.74 秒
音频时长已写入文件: /home/renmengxing/audioRec/audio_data/test_with_duration.csv



时长计算完成！
成功处理音频文件数量: 9473 / 9473
最短音频时长: 0.30 秒
最长音频时长: 30.00 秒

In [3]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [5]:
print(121)

121
